In [22]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_experimental.agents import create_pandas_dataframe_agent
from dotenv import load_dotenv
import os

In [23]:
url="https://www.harighotra.co.uk/indian-recipes/time/indian-recipes-in-30-60-minutes/view-all"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')


In [24]:
articles = soup.find_all('article')

In [25]:
recipe=[]
for a in articles:
    name_tag= a.find("h3")
    time_tag= a.find('h4',attrs={"content":True})
    if name_tag and time_tag:
        name = name_tag.text.strip()
        time = time_tag.text.strip()
        recipe.append([name, time])

In [26]:
with open('data.csv','w',newline='',encoding='utf-8')as file:
    writer=csv.writer(file)
    writer.writerow(['name','time'])
    writer.writerows(recipe)

In [27]:
df=pd.read_csv('data.csv')

In [28]:
df['time'].head(10)

0            1 hr
1         50 mins
2            1 hr
3         30 mins
4         50 mins
5    1 hr 20 mins
6         40 mins
7         50 mins
8         50 mins
9         40 mins
Name: time, dtype: object

In [29]:
def time_min(time_str):
    time_str=str(time_str).lower().strip()
    min=0
    if 'hr' in time_str:
        hr_part=time_str.split('hr')[0].strip()
        if hr_part.isdigit():
            min+=int(hr_part)*60
    if 'min' in time_str:
        min_part=''.join(filter(str.isdigit,time_str.split('hr')[-1]))
        if min_part:
            min+=int(min_part)
    return min

In [30]:
df['time_min']=df['time'].apply(time_min)

In [31]:
df.head(10)

,name,time,time_min
0,Pistachio chicken,1 hr,60
1,Turkey Biryani,50 mins,50
2,Healthy Chicken Korma,1 hr,60
3,Brown Lentil Dhal,30 mins,30
4,Thari Wala Chicken,50 mins,50
5,Lamb Dhansak,1 hr 20 mins,80
6,Chicken Pasanda,40 mins,40
7,Achaari Bangun,50 mins,50
8,Aloo Bangun,50 mins,50
9,Bombay Aloo,40 mins,40


In [40]:
min_time=30

In [39]:
filtered_df=df[df['time_min']<=min_time]
filtered_df

,name,time,time_min
3,Brown Lentil Dhal,30 mins,30
10,Coastal Squid Curry,25 mins,25
12,Cabbage Thoran,30 mins,30
13,Chilli Paneer Recipe,20 mins,20
15,Aloo Parantha,30 mins,30
16,Aloo Tama Bodi,30 mins,30
17,Aloo Tikki,30 mins,30
18,Amchoor Stuffed Bangun,30 mins,30
35,Cockle Curry,20 mins,20
37,Crispy Hot Wings,30 mins,30


In [ ]:
langchain_key = os.getenv("LANGCHAIN_API_KEY")
langsmith_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

In [33]:
load_dotenv()
os.environ["LANGCHAIN_API_KEY"] = langchain_key
os.environ["LANGSMITH_API_KEY"] = langsmith_key
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
LANGSMITH_PROJECT="meal_bot"

TypeError: str expected, not NoneType

In [ ]:

# Connect to Gemma (via Ollama)
llm = Ollama(model='gemma3')
# Create LangChain agent that can read the dataframe
agent = create_pandas_dataframe_agent(llm, df, allow_dangerous_code=True, verbose=True)

# Ask your question
response = agent.run(
    "Pick 3 different meals for today: one breakfast, one lunch, one dinner. "
    "Choose randomly and return them in a table with Meal, Dish Name, and Time (mins)."
)

print(response)







> Entering new AgentExecutor chain...
Thought: I need to select three different meals from the dataframe and create a table with the meal name, dish name and time in minutes. I will use the `python_repl_ast` to generate random rows from the dataframe and create a table with the required columns.
Action: python_repl_ast
Action Input: `import pandas as pd; df.sample(3)`                            name     time  time_min
21   Beetroot and Coconut Samosa  40 mins        40
28             Chicken Balti Pie     1 hr        60
78  Paneer with Mixed Vegetables  40 mins        40I have three different meals: Beetroot and Coconut Samosa, Chicken Balti Pie and Paneer with Mixed Vegetables. Now I will format the result in a table with the columns Meal, Dish Name and Time (mins).
Action: python_repl_ast
Action Input: `import pandas as pd; df.sample(3)`                             name     time  time_min
89                    Saag Paneer  50 mins        50
100  Spicy Lemon Roasted Potatoes  55 min